In [ ]:
import polars as pl
pl.Config(fmt_str_lengths=200)

In [ ]:
import os
os.getcwd()

In [ ]:
test = pl.read_csv('/home/jovyan/smirnov/our_submissions/test_dataset_submission_queries.csv')

In [ ]:
test.head(5)

In [ ]:
import txtai
embeddings = txtai.Embeddings()
embeddings.load("/home/jovyan/smirnov/our_indexes/all_videos")

In [ ]:
embeddings.search('Экстрасенсы. Битва сильнейших, 4 выпуск',5)

In [ ]:
q = 'Экстрасенсы. Битва сильнейших, 4 выпуск 2023'
limit=5
embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q}") \
order by score desc, v_pub_datetime desc \
limit {limit}')

In [ ]:
len(test)

In [ ]:
import re
submission={"query" :[], "video_id":[]}
for q in test['query']:
    #print(q)
    q_clean=re.sub("[^а-яА-ЯЁё0-9a-zA-Z ]","",q)
    #print(q)
    limit=5
    res = embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q_clean}") \
    order by score desc \
    limit {limit}')
    submission['query'] +=[q]*len(res)
    submission['video_id'] +=(list(map(lambda x: x['id'],res)))

len(res)

In [19]:
import nltk 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('popular')
# Очистка текста (пример функции)
def clean_text(text):
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    tokens = nltk.word_tokenize(text)
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stopwords.words('russian')]
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

In [ ]:
len(test)

## Удаление стоп слов

In [20]:
import re
submission = {"query": [], "video_id": []}
limit = 5  # Fixed number of results per query
for q in test['query']:
    q_clean = clean_text(re.sub("[^а-яА-ЯЁё0-9a-zA-Z ]", "", q))

    # Execute search
    res = embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q_clean}") \
                            order by score desc \
                            limit {limit}')

    # If there are results, extend the submission dictionary with these results
    submission['query'] += [q] * 5
    submission['video_id'] += [x['id'] for x in res]

    # If the results are fewer than the limit, fill the remaining slots with empty values
    if len(res) < limit:
        submission['video_id'] += [None] * (limit - len(res))

In [21]:
len(submission['query'])

10000

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 2000)

In [ ]:
len(submission)

In [ ]:
sub_df = pd.DataFrame(submission)
sub_df.head(10)

In [22]:
sub_df.to_csv('/home/jovyan/smirnov/our_submissions/txtai_all_submission_03_without_order_with_stop.csv')

## Очистка стоп слов

In [ ]:
import re
submission={"query" :[], "video_id":[]}
for q in test['query'].head(20):
    #print(q)
    q_clean=re.sub("[^а-яА-ЯЁё0-9a-zA-Z ]","",q)
    print(q_clean)
    limit=1
    res = embeddings.search(f'select id, text, v_pub_datetime, score from txtai where similar ("{q_clean}") \
    order by score desc \
    limit {limit}')
    submission['query'] +=[q]*len(res)
    submission['video_id'] +=(list(map(lambda x: x['id'],res)))
    #print(res)